# LLM - Klasyczne metody klasyfikacji tekstu - LAB

# Zadanie

Zaadaptuj kod z notatnika *LLM - Klasyczne metody klasyfikacji tekstu - Omówienie* do problemu klasyfikacji liczby gwiazdek dla opinii z serwisu Yelp.
Możesz przygotować pętlę treningową albo w czystym PyTorchu, albo z wykorzystaniem biblioteki PyTorch Lightning.

* Wykorzystaj zbiór `Yelp/yelp_review_full` ([link](https://huggingface.co/datasets/Yelp/yelp_review_full)) zawierający opinie z serwisu Yelp (kolumna: `text`) i etykietę (kolumna: `label`) o wartościach $0,1,2,3,4$ określającą liczbę gwiazdek przyznaną przez użytkownika (a ściślej, liczbę gwiazdek minus jeden). Ponieważ mamy pięć klas, ostatnia warstwa liniowa w sieci neuronowej musi zwracać pięć wartości.
    * Zgodnie z dobrą praktyką z części treningowej wydziel dodatkową część walidacyjną.
    * (opcjonalnie) Ogranicz rozmiar każdej części zbioru danych (treningowej, walidacyjnej i testowej). Część treningowa nie powinna zawierać więcej niż 100k elementów.
* Do ekstrakcji cech z tekstu wykorzystaj **metodę TF-IDF** (*term frequency-inverse document frequency*) opartą o podejście typu worek słów (*bag-of-words*). Zastosuj funkcję `TfidfVectorizer` z biblioteki `scikit-learn`.


## Punkty do wykonania

1.   Napisz funkcję znajdującą i wyświetlającą $k$ elementów zbioru testowego dla których model najbardziej się myli, czyli predykuje najmniejsze prawdopodobieństwa prawdziwej klasy. Softmax jest funkcją ściśle rosnącą, więc wystarczy znaleźć elementy z najmniejszą wartością nieznormalizowanego wyjścia z sieci (logita) dla prawdziwej klasy.
2.   Zbadaj wpływ wybranych parametrów funkcji ekstrakcji cech z tekstu `TfidfVectorizer` na skuteczność wytrenowanego modelu. Uruchom kilka eksperymentów z różnymi wartościami parametrów i porównaj dokładność wytrenowanego modelu na zbiorze walidacyjnym.
3.   Zbadaj wpływ wybranych hiperparametrów modelu (np. liczba warstw liniowych modelu, rozmiary warstw) i procesu uczenia (np. początkowa wartość stopy uczenia, liczba epok, typ i parametry planisty stopy uczenia, typ i parametry optymalizatora) na skuteczność wytrenowanego modelu. Uruchom kilka eksperymentów z różnymi wartościami hiperparametrów i porównaj dokładność wytrenowanego modelu na zbiorze walidacyjnym. Następnie wykonaj finalną ewaluację najlepszego modelu na zbiorze testowym.


Import bibliotek.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import heapq
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from datasets import load_dataset
from sklearn.metrics import (
    f1_score,
    classification_report,
    recall_score,
    accuracy_score,
    precision_score,
    confusion_matrix,
)

print(f"Wersja biblioteki PyTorch: {torch.__version__}")

Wersja biblioteki PyTorch: 2.9.0+cu128


/home/atarsander/Programs/anaconda3/envs/LLM/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/atarsander/Programs/anaconda3/envs/LLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sprawdzenie dostępności GPU.

In [2]:
print(f"Dostępność GPU: {torch.cuda.is_available()}")
print(f"Typ GPU: {torch.cuda.get_device_name(0)}")


Dostępność GPU: True
Typ GPU: NVIDIA GeForce RTX 5070 Ti


In [3]:
import wandb

# Logowanie do serwisu Weights&Biases monitorującego przebieg eksperymentów
wandb.login(key="b18357d829db3e608dce0a0b0637312f25532350")

/home/atarsander/Programs/anaconda3/envs/LLM/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/atarsander/Programs/anaconda3/envs/LLM/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This m

True

# Rozwiązanie

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
NUM_WORKERS = 8

In [5]:
dataset_train = load_dataset("Yelp/yelp_review_full", split="train[:100000]")
dataset_test = load_dataset("Yelp/yelp_review_full", split="test[:20000]")

In [6]:
ds = dataset_train.train_test_split(test_size=0.15)
dataset_train, dataset_val = ds["train"], ds["test"]

In [7]:
print(f"Train dataset size: {len(dataset_train)}")
print(f"Validation dataset size: {len(dataset_val)}")
print(f"Test dataset size: {len(dataset_test)}")

Train dataset size: 85000
Validation dataset size: 15000
Test dataset size: 20000


In [ ]:
class MLP(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList()
        for layer in layers:
            if "dropout" in layer:
                self.layers.append(nn.Dropout(layer["dropout"]))
            if "linear" in layer:
                self.layers.append(nn.Linear(*layer["linear"]))
            if "batch_norm" in layer:
                self.layers.append(nn.BatchNorm1d(layer["batch_norm"]))
            if "relu" in layer:
                self.layers.append(nn.ReLU())
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x  

In [9]:
def calc_accuracy(y_pred, y):
    return (y_pred == y).sum().item() / len(y)

In [ ]:
vectorizer = TfidfVectorizer(max_features=10000, min_df=2, max_df=0.95)
X_train_tf_idf = vectorizer.fit_transform(dataset_train["text"])
X_val_tf_idf = vectorizer.transform(dataset_val["text"])
X_test_td_idf = vectorizer.transform(dataset_test["text"])

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,analyzer,'word'
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"


In [ ]:
svd = TruncatedSVD(n_components=300)
X_train = svd.fit_transform(X_train_tf_idf)
X_val = svd.transform(X_val_tf_idf)
X_test = svd.transform(X_test_td_idf)

In [ ]:
y_train = np.array(dataset_train["label"])
y_val = np.array(dataset_val["label"])
y_test = np.array(dataset_val["label"])

In [ ]:
train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
val_dataset = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

In [13]:
def _to_dense_tensor(sparse_mat, device):
    return torch.from_numpy(sparse_mat.toarray()).to(device=device, dtype=torch.float32)

In [ ]:
def predict(model, loader, device):
    logits_list = []
    label_list = []
    with torch.inference_mode():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            logits = model(X)
            logits_list.extend(logits.detach().cpu().numpy())
            label_list.extend(y)
    y_preds = np.argmax(logits_list, axis=1)
    return y_preds, label_list

In [ ]:
def train(
    train_loader,
    eval_loader,
    model,
    epochs,
    optim_type,
    optim_params,
    criterion,
    device
):
    
    optimizer = optim_type(model.parameters(), *optim_params)
    losses = []
    for epoch in range(epochs):
        model.train()
        avg_loss = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
        losses.append(avg_loss / len(train_loader))
        if epoch % 5 == 0:
            model.eval()
            train_pred_y, train_y = predict(model, train_loader, device)
            train_acc = calc_accuracy(train_pred_y, train_y)
            val_pred_y, val_y = predict(model, eval_loader, device)
            val_acc = calc_accuracy(val_pred_y, val_y)

            print(
                f"Epoch: {epoch}      Loss: {avg_loss:.5f} | train accuracy: {train_acc:.4f} | validation accuracy: {val_acc:.4f}"
            )
    return losses

In [16]:
def find_k_smallest(model, loader, K, vectorizer, device):
    k_smallest_list = []
    model.eval()
    with torch.inference_mode():
        for text, label in loader:
            X = vectorizer.fit_transform(text)
            X, label = X.to(device), label.to(device)
            logits = model(X)
            heapq.heappush(k_smallest_list, (-logits.min(dim=1), logits.argmin(dim=1)))
            if len(k_smallest_list) > K:
                heapq.heappop(k_smallest_list)
    return k_smallest_list

In [17]:
def tune_architecture(
    model_class, train_loader, val_loader, architectures, training_setup, device
):
    losses = []
    train_accuracy = []
    val_accuracy = []
    train_precision = []
    val_precision = []
    train_recall = []
    val_recall = []
    train_f1 = []
    val_f1 = []
    train_confusion_matrix = []
    val_confusion_matrix = []

    for i, architecture in enumerate(architectures):
        model = model_class(architecture).to(device)
        print(f"Training model {i}")
        loss = train(
            train_loader,
            val_loader,
            model,
            training_setup["epochs"],
            training_setup["optim_type"],
            training_setup["optim_params"],
            training_setup["criterion"],
            vectorizer=training_setup["vectorizer"],
            device=device
        )
        print("Training done")
        losses.append(loss)
        train_y_preds, train_y = predict(model, train_loader, training_setup["vectorizer"], device)
        train_accuracy.append(accuracy_score(train_y, train_y_preds))
        train_precision.append(precision_score(train_y, train_y_preds, average="macro"))
        train_recall.append(recall_score(train_y, train_y_preds, average="macro"))
        train_f1.append(f1_score(train_y, train_y_preds, average="macro"))
        train_confusion_matrix.append(confusion_matrix(train_y, train_y_preds))

        val_y_preds, val_y = predict(model, val_loader, training_setup["vectorizer"], device)
        val_accuracy.append(accuracy_score(val_y, val_y_preds))
        val_precision.append(precision_score(val_y, val_y_preds, average="macro"))
        val_recall.append(recall_score(val_y, val_y_preds, average="macro"))
        val_f1.append(f1_score(val_y, val_y_preds, average="macro"))
        val_confusion_matrix.append(confusion_matrix(val_y, val_y_preds))
        print(classification_report(val_y, val_y_preds))

    return (
        losses,
        train_accuracy,
        train_precision,
        train_recall,
        train_f1,
        train_confusion_matrix,
        val_accuracy,
        val_precision,
        val_recall,
        val_f1,
        val_confusion_matrix,
    )

### Porównanie różnych architektur

In [ ]:
architectures = [
    [
        {"linear": (10000, 512), "relu": True},
        {"linear": (512, 256), "relu": True},
        {"linear": (256, 5)},
    ],
    [
        {"linear": (10000, 256), "relu": True},
        {"linear": (256, 128), "relu": True},
        {"linear": (128, 5)},
    ],
    [
        {"linear": (10000, 1024), "relu": True},
        {"linear": (1024, 512), "relu": True},
        {"linear": (512, 256), "relu": True},
        {"linear": (256, 5)},
    ],
    [
        {"linear": (10000, 512), "relu": True},
        {"dropout": 0.3, "linear": (512, 256), "relu": True},
        {"dropout": 0.3, "linear": (256, 5)},
    ],
    [
        {"linear": (10000, 256), "relu": True},
        {"dropout": 0.3, "linear": (256, 128), "relu": True},
        {"dropout": 0.3, "linear": (128, 5)},
    ],
    [
        {"linear": (10000, 512), "relu": True},
        {"dropout": 0.3, "linear": (512, 256), "batch_norm": 256, "relu": True},
        {"dropout": 0.3, "linear": (256, 128), "batch_norm": 128, "relu": True},
        {"dropout": 0.3, "linear": (128, 5)},
    ],
]

In [19]:
training_setup = {
    "epochs": 40,
    "optim_type": torch.optim.Adam,
    "optim_params": [3e-4],
    "criterion": nn.CrossEntropyLoss(),
    "vectorizer": vectorizer,
}

In [20]:
(
    losses,
    train_accuracy,
    train_precision,
    train_recall,
    train_f1,
    train_confusion_matrix,
    val_accuracy,
    val_precision,
    val_recall,
    val_f1,
    val_confusion_matrix,
) = tune_architecture(MLP, train_loader, val_loader, architectures, training_setup, device=DEVICE)

Training model 0
Epoch: 0      Loss: 2722.47147 | train accuracy: 0.7240 | validation accuracy: 0.5879
Epoch: 5      Loss: 247.34829 | train accuracy: 0.9937 | validation accuracy: 0.5239
Epoch: 10      Loss: 138.49409 | train accuracy: 0.9968 | validation accuracy: 0.5143
Epoch: 15      Loss: 99.23743 | train accuracy: 0.9981 | validation accuracy: 0.5232
Epoch: 20      Loss: 79.15661 | train accuracy: 0.9987 | validation accuracy: 0.5097
Epoch: 25      Loss: 61.02388 | train accuracy: 0.9990 | validation accuracy: 0.5191
Epoch: 30      Loss: 56.27738 | train accuracy: 0.9990 | validation accuracy: 0.5116
Epoch: 35      Loss: 52.64089 | train accuracy: 0.9991 | validation accuracy: 0.5208
Training done
              precision    recall  f1-score   support

           0       0.66      0.70      0.68      3472
           1       0.42      0.47      0.44      2979
           2       0.42      0.37      0.39      2949
           3       0.42      0.38      0.40      2957
           4    

## 1. 

## 2.

## 3.